In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\| **Quickstart** \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Quickstart 快速入门
==========

This section runs through the API for common tasks in machine learning.
Refer to the links in each section to dive deeper.

本节介绍了用于机器学习常见任务的API。请参考每节中的链接以深入了解。

Working with data 
-----------------

PyTorch has two [primitives to work with
data](https://pytorch.org/docs/stable/data.html):
`torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. `Dataset`
stores the samples and their corresponding labels, and `DataLoader`
wraps an iterable around the `Dataset`.

PyTorch有两个用于处理数据的[基本组件](https://pytorch.org/docs/stable/data.html)：`torch.utils.data.DataLoader`和`torch.utils.data.Dataset`。`Dataset`存储样本及其对应的标签，而`DataLoader`在`Dataset`周围包装了一个可迭代对象。

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), all of which
include datasets. For this tutorial, we will be using a TorchVision
dataset.

PyTorch提供了领域特定的库，如[TorchText](https://pytorch.org/text/stable/index.html)、[TorchVision](https://pytorch.org/vision/stable/index.html)和[TorchAudio](https://pytorch.org/audio/stable/index.html)，这些库都包含数据集。在本教程中，我们将使用一个TorchVision数据集。

The `torchvision.datasets` module contains `Dataset` objects for many
real-world vision data like CIFAR, COCO ([full list
here](https://pytorch.org/vision/stable/datasets.html)). In this
tutorial, we use the FashionMNIST dataset. Every TorchVision `Dataset`
includes two arguments: `transform` and `target_transform` to modify the
samples and labels respectively.

`torchvision.datasets`模块包含许多实际视觉数据的`Dataset`对象，例如CIFAR、COCO（[完整列表在此](https://pytorch.org/vision/stable/datasets.html)）。在本教程中，我们使用FashionMNIST数据集。每个TorchVision `Dataset`包括两个参数：`transform`和`target_transform`，用于分别修改样本和标签。

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an
iterable over our dataset, and supports automatic batching, sampling,
shuffling and multiprocess data loading. Here we define a batch size of
64, i.e. each element in the dataloader iterable will return a batch of
64 features and labels.

我们将`Dataset`作为参数传递给`DataLoader`。`DataLoader`在我们的数据集上包装了一个可迭代对象，并支持自动批处理、采样、混洗和多进程数据加载。在这里，我们定义了一个批大小为64，即`dataloader`可迭代对象中的每个元素将返回一个包含64个特征和标签的批次。

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Read more about [loading data in PyTorch](data_tutorial.html).

阅读更多关于[在PyTorch中加载数据](data_tutorial.html)的信息。

------------------------------------------------------------------------


Creating Models
===============

To define a neural network in PyTorch, we create a class that inherits
from
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
We define the layers of the network in the `__init__` function and
specify how data will pass through the network in the `forward`
function. To accelerate operations in the neural network, we move it to
the GPU or MPS if available.

要在PyTorch中定义一个神经网络，我们创建一个继承自[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的类。我们在`__init__`函数中定义网络的层，并在`forward`函数中指定数据如何通过网络传递。为了加速神经网络中的操作，我们将其移动到GPU或MPS（如果可用）。

MPS（Metal Performance Shaders）是Apple开发的一组高性能图形和计算着色器，用于其设备上的GPU加速。MPS允许在Mac和iOS设备上使用Metal API进行高效的机器学习计算和图像处理。在PyTorch中，MPS支持可以加速在Apple设备（如MacBook、iMac和iPad）上的深度学习模型训练和推理。

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in
PyTorch](buildmodel_tutorial.html).

阅读更多关于[在PyTorch中构建神经网络](buildmodel_tutorial.html)的信息。

------------------------------------------------------------------------


Optimizing the Model Parameters
===============================

To train a model, we need a [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).

要训练一个模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)和一个[优化器](https://pytorch.org/docs/stable/optim.html)。

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training
dataset (fed to it in batches), and backpropagates the prediction error
to adjust the model\'s parameters.

在一个训练循环中，模型对训练数据集进行预测（以批次形式输入），并通过反向传播预测误差来调整模型的参数。

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model\'s performance against the test dataset to
ensure it is learning.

我们还检查模型在测试数据集上的性能，以确保它正在学习。

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*).
During each epoch, the model learns parameters to make better
predictions. We print the model\'s accuracy and loss at each epoch;
we\'d like to see the accuracy increase and the loss decrease with every
epoch.

训练过程在多个迭代（epochs）中进行。在每个epoch中，模型学习参数以做出更好的预测。我们在每个epoch打印模型的准确性和损失；我们希望看到准确性增加和损失减少。

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.288214  [   64/60000]
loss: 2.287361  [ 6464/60000]
loss: 2.260547  [12864/60000]
loss: 2.264386  [19264/60000]
loss: 2.243507  [25664/60000]
loss: 2.214134  [32064/60000]
loss: 2.227824  [38464/60000]
loss: 2.178689  [44864/60000]
loss: 2.184913  [51264/60000]
loss: 2.161205  [57664/60000]
Test Error: 
 Accuracy: 44.6%, Avg loss: 2.147166 

Epoch 2
-------------------------------
loss: 2.152851  [   64/60000]
loss: 2.144938  [ 6464/60000]
loss: 2.078959  [12864/60000]
loss: 2.093534  [19264/60000]
loss: 2.038022  [25664/60000]
loss: 1.981472  [32064/60000]
loss: 2.014562  [38464/60000]
loss: 1.917836  [44864/60000]
loss: 1.935759  [51264/60000]
loss: 1.862307  [57664/60000]
Test Error: 
 Accuracy: 54.1%, Avg loss: 1.856269 

Epoch 3
-------------------------------
loss: 1.894232  [   64/60000]
loss: 1.860893  [ 6464/60000]
loss: 1.735348  [12864/60000]
loss: 1.772067  [19264/60000]
loss: 1.670075  [25664/60000]
loss: 1.628636  [32064/600

Read more about [Training your model](optimization_tutorial.html).

阅读更多关于[训练你的模型](optimization_tutorial.html)的信息。

------------------------------------------------------------------------


Saving Models
=============

A common way to save a model is to serialize the internal state
dictionary (containing the model parameters).

保存模型的常见方法是序列化内部状态字典（包含模型参数）。

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
==============

The process for loading a model includes re-creating the model structure
and loading the state dictionary into it.


加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [11]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

现在可以使用该模型进行预测。

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Read more about [Saving & Loading your
model](saveloadrun_tutorial.html).

阅读更多关于[保存和加载模型](saveloadrun_tutorial.html)的信息。